In [1]:
import torch, torch.nn as nn
from sklearn.model_selection import train_test_split
import torch
import csv
from Metrics import spearman_correlation, KendallTau, PearsonCorrelation, RMSE
from ReadData import readJsonData
from LSTM_MLP_Model import LSTM_MLP_Model
from GetDataTensors import getDataTensors
from LossFunction import CustomLoss
from Parameters import training_show_every as show_every, training_epochs as epochs

In [2]:
csv_fields = ['epoch', 'train_loss', 'test_loss', 'person', 'person_p', 'spearman', 'spearman_p', 'rmse']
csv_file_name = 'LSTM-MLP-Results.csv'

csv_data_points_fields = ['real', 'predicted']
csv_data_points_file_name = 'LSTM-MLP-Data-Points.csv'

preprocessed_data, number_of_videos = readJsonData('./data.json')
        
print("Number of videos: ", number_of_videos)

features_tensor = torch.tensor([item[3] for item in preprocessed_data])
importance_tensor = torch.tensor([item[2] for item in preprocessed_data]).unsqueeze(1)

Number of videos:  50


In [3]:
with open(csv_file_name, mode='w', newline='') as csv_file, open(csv_data_points_file_name, mode='w', newline='') as csv_data_points_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_fields)
    csv_data_points_writer = csv.writer(csv_data_points_file)
    csv_data_points_writer.writerow(csv_data_points_fields)
    for lmbda in [1.15 + (0.05 * i) for i in range(1)]:
        csv_writer.writerow([f'Lambda: {lmbda}'])
        print(f'Lambda: {lmbda}')
        
        model = LSTM_MLP_Model()

        # Loss and optimizer
        criterion = CustomLoss(lmbda)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

        # Get the data tensors
        features_training_tensor, features_testing_tensor, importance_training_tensor, importance_testing_tensor = getDataTensors(features_tensor, importance_tensor)

        # Train the model (dummy example, replace with your actual data)
        features_training_tensor.requires_grad_()
        
        for epoch in range(1, epochs+1):
            outputs = model(features_training_tensor)
            targets = importance_training_tensor  # Targets are the same as input for reconstruction
            loss = criterion(outputs.view(-1,1), targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                    test_outputs = model(features_testing_tensor)
                    test_loss = criterion(test_outputs.view(-1,1), importance_testing_tensor)
            
            pearson, pearson_p = PearsonCorrelation(test_outputs.view(-1), importance_testing_tensor.view(-1))
            spearman, spearman_p = spearman_correlation(test_outputs.view(-1), importance_testing_tensor.view(-1))
            rmse = RMSE(test_outputs.view(-1), importance_testing_tensor.view(-1))
        
            csv_writer.writerow([epoch, loss.item(), test_loss.item(), pearson, pearson_p, spearman, spearman_p, rmse.item()])
            
            if epoch % show_every == 0 or epoch == 1:
                print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Pearson: {pearson:.4f}, Pearson p: {pearson_p:.4f}, Spearman: {spearman:.4f}, Spearman p: {spearman_p:.4f}, RMSE: {rmse:.4f}')
                if test_loss.item() > prev_loss:
                    print("Overfitting...")
                    break
                else:
                    prev_loss = test_loss.item()
        
        data_points = list(zip(importance_testing_tensor.view(-1).tolist(), test_outputs.view(-1).tolist()))
        csv_data_points_writer.writerow(f'Lambda: {lmbda}')
        csv_data_points_writer.writerows(data_points)
        # Save the model
        torch.save(model.state_dict(), f'LSTM-MLP-Lambda-{lmbda}.pth')

Lambda: 1.15
Epoch [1/1000], Loss: 2.8431, Test Loss: 3.0485, Pearson: 0.0786, Pearson p: 0.0000, Spearman: 0.0840, Spearman p: 0.0000, RMSE: 1.9469
Epoch [25/1000], Loss: 2.5639, Test Loss: 2.8693, Pearson: 0.1341, Pearson p: 0.0000, Spearman: 0.1137, Spearman p: 0.0000, RMSE: 1.9005
Epoch [50/1000], Loss: 2.3291, Test Loss: 2.6354, Pearson: 0.1194, Pearson p: 0.0000, Spearman: 0.1016, Spearman p: 0.0000, RMSE: 1.6368
Epoch [75/1000], Loss: 1.2916, Test Loss: 1.3649, Pearson: 0.0953, Pearson p: 0.0000, Spearman: 0.0921, Spearman p: 0.0000, RMSE: 1.2275
Epoch [100/1000], Loss: 0.7135, Test Loss: 0.9157, Pearson: 0.1008, Pearson p: 0.0000, Spearman: 0.1043, Spearman p: 0.0000, RMSE: 1.6668
Epoch [125/1000], Loss: 0.6483, Test Loss: 0.8784, Pearson: 0.1193, Pearson p: 0.0000, Spearman: 0.1154, Spearman p: 0.0000, RMSE: 1.6585
Epoch [150/1000], Loss: 0.6031, Test Loss: 0.8668, Pearson: 0.1254, Pearson p: 0.0000, Spearman: 0.1175, Spearman p: 0.0000, RMSE: 1.6387
Epoch [175/1000], Loss: 0.

In [7]:
kendall, kendall_p = KendallTau(importance_testing_tensor.view(-1), test_outputs.view(-1))
print(f'Kendall: {kendall:.4f}, Kendall p: {kendall_p:.4f}')

Kendall: 0.0818, Kendall p: 0.0000


In [24]:
for lmbda in [0.5 + (0.5 * i) for i in range(4)]:
    for mu in [0.5 + (0.5 * i) for i in range(4)]:
        print(f'Lambda: {lmbda}, Mu: {mu}')
        with open(f'./VsLossParameters/Points_lambda_{lmbda}_mu_{mu}.csv', mode='w', newline='') as csv_file, open(f'./VsLossParameters/Results_lambda_{lmbda}_mu_{mu}.csv', mode='w', newline='') as csv_data_points_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(csv_fields)
            csv_data_points_writer = csv.writer(csv_data_points_file)
            csv_data_points_writer.writerow(csv_data_points_fields)

            pearson = 0
            rmse = 3
            
            torch.manual_seed(6)
            
            prev_loss = 1000000
            model = LSTM_MLP_Model()

            # Loss and optimizer
            criterion = CustomLoss(lmbda, mu)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
            # Get the data tensors
            features_training_tensor, features_testing_tensor, importance_training_tensor, importance_testing_tensor = getDataTensors(features_tensor, importance_tensor)

            # Train the model (dummy example, replace with your actual data)
            features_training_tensor.requires_grad_()
            
            for epoch in range(1, epochs+1):
                outputs = model(features_training_tensor)
                targets = importance_training_tensor  # Targets are the same as input for reconstruction
                loss = criterion(outputs.view(-1,1), targets)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                with torch.no_grad():
                    test_outputs = model(features_testing_tensor)
                    test_loss = criterion(test_outputs.view(-1,1), importance_testing_tensor)
                
                pearson, pearson_p = PearsonCorrelation(test_outputs.view(-1), importance_testing_tensor.view(-1))
                spearman, spearman_p = spearman_correlation(test_outputs.view(-1), importance_testing_tensor.view(-1))
                rmse = RMSE(test_outputs.view(-1), importance_testing_tensor.view(-1))
            
                csv_writer.writerow([epoch, loss.item(), test_loss.item(), pearson, pearson_p, spearman, spearman_p, rmse.item()])
                
                if epoch % show_every == 0 or epoch == 1:
                    #if epoch == 1: print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Pearson: {pearson:.4f}, Pearson p: {pearson_p:.4f}, Spearman: {spearman:.4f}, Spearman p: {spearman_p:.4f}, RMSE: {rmse:.4f}')
                    print(f"{epoch}", end=", ")
                    if test_loss.item() > prev_loss:
                        print("Overfitting...")
                        break
                    else:
                        prev_loss = test_loss.item()
            print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Pearson: {pearson:.4f}, Pearson p: {pearson_p:.4f}, Spearman: {spearman:.4f}, Spearman p: {spearman_p:.4f}, RMSE: {rmse:.4f}')
            data_points = list(zip(importance_testing_tensor.view(-1).tolist(), test_outputs.view(-1).tolist()))
            csv_data_points_writer.writerows(data_points)
            # Save the model
            torch.save(model.state_dict(), f'./VsLossParameters/Model_lambda_{lmbda}_mu_{mu}.pth')

Lambda: 0.5, Mu: 0.5
1, 10, 20, 30, Overfitting...
Epoch [30/1000], Loss: 0.4694, Test Loss: 0.6750, Pearson: 0.0855, Pearson p: 0.0000, Spearman: 0.0788, Spearman p: 0.0000, RMSE: 1.4368
Lambda: 0.5, Mu: 1.0
1, 10, 20, 30, Overfitting...
Epoch [30/1000], Loss: 0.6161, Test Loss: 1.0462, Pearson: 0.0813, Pearson p: 0.0000, Spearman: 0.0590, Spearman p: 0.0001, RMSE: 1.4612
Lambda: 0.5, Mu: 1.5
1, 10, 20, 30, 40, Overfitting...
Epoch [40/1000], Loss: 0.6497, Test Loss: 1.3829, Pearson: 0.0861, Pearson p: 0.0000, Spearman: 0.0609, Spearman p: 0.0000, RMSE: 1.4031
Lambda: 0.5, Mu: 2.0
1, 10, 20, 30, 40, Overfitting...
Epoch [40/1000], Loss: 0.7401, Test Loss: 1.8315, Pearson: 0.0724, Pearson p: 0.0000, Spearman: 0.0456, Spearman p: 0.0018, RMSE: 1.3589
Lambda: 1.0, Mu: 0.5
1, 10, 20, 30, 40, Overfitting...
Epoch [40/1000], Loss: 0.4479, Test Loss: 0.7029, Pearson: 0.0711, Pearson p: 0.0000, Spearman: 0.0649, Spearman p: 0.0000, RMSE: 1.4765
Lambda: 1.0, Mu: 1.0
1, 10, 20, 30, 40, Overfitt